In [76]:

import torch
import torch.nn as nn
import numpy as np
from copy import deepcopy

device = "cuda" if torch.cuda.is_available() else "cpu"

class RBFlayer(nn.Module):
    def __init__(self, timelag):
        super(RBFlayer, self).__init__()

        self.timelag = timelag

        device = "cuda" if torch.cuda.is_available() else "cpu"
        torch.cuda.manual_seed(0)

        self.init_weight_cause = nn.Parameter(torch.rand(self.timelag, device=device))
        self.init_weight_target = nn.Parameter(torch.rand(self.timelag, device=device))
        self.cause_clt  = self.init_clt()
        self.cause_std = self.init_clt()
        self.target_clt = nn.Parameter(torch.rand(1,device=device))
        self.target_std = nn.Parameter(torch.rand(1, device=device))

    def init_clt(self):
        return nn.Parameter(torch.rand(1,device=device))

    def init_std(self):
        return nn.Parameter(torch.rand(1, device=device))

    def rbf(self, x, cluster, std):
        return torch.exp(-(x - cluster) * (x - cluster) / 2*(std * std))

    def forward(self, cause, target):
        
        for i in range(len(cause)):
            if i == 0:
                a = self.rbf(cause[i], self.cause_clt, self.cause_std)
            else:
                a = torch.cat([a,self.rbf(cause[i], self.cause_clt, self.cause_std)],dim = 0)
        cause = self.init_weight_cause * a
        
        for j in range(len(target)):
            if j == 0:
                b = self.rbf(target[j], self.target_clt, self.target_std)
            else:
                b = torch.cat([b,self.rbf(target[j], self.target_clt, self.target_std)],dim = 0)
        target = self.init_weight_target * b
        
        return cause, target


class RBFnet(nn.Module):
    def __init__(self, input_size , output_size, timelag):
        super(RBFnet,self).__init__()
    
        self.input_size = input_size      # number of data
        self.output_size = output_size
        self.timelag = timelag

        self.linear = nn.ModuleList([nn.Linear(self.timelag*2,1) for _ in range(self.input_size)])
        self.relu = nn.ReLU()
        self.networks = nn.ModuleList([RBFlayer(self.timelag) for _ in range(self.input_size)])

    def cause_target(self, cause, target):
        x = torch.cat((cause, target), 0)

        return x

    def GC(self, threshold=True):
        '''
        Extract learned Granger causality.
        Args:
          threshold: return norm of weights, or whether norm is nonzero.
        Returns:
          GC: (p x p) matrix. Entry (i, j) indicates whether variable j is
            Granger causal of variable i.
        '''
        GC = [torch.norm(net.init_weight_cause, dim=0)
              for net in self.networks]
        GC = torch.stack(GC)
        if threshold:
            return (GC > 0).int()
        else:
            return GC


    def forward(self, causes, targets):
        out_list = []
        for i in range(self.input_size):
            cause, target = self.networks[i](causes[i], targets[i])
            cause, target = self.relu(cause), self.relu(target)
            pred = torch.cat((cause, target),0)
            pred = self.linear[i](pred)
            out_list.append(pred)

        return out_list


def restore_parameters(model, best_model):
    '''Move parameter values from best_model to model.'''
    for params, best_params in zip(model.parameters(), best_model.parameters()):
        params.data = best_params


def train_rbf(model, input_causes, input_targets, Y, lr, epochs, lookback=5,device = device):
    # input_causes, input_targets : X
    # Y : Y
    model.to(device)
    loss_fn = nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_loss_list = []

    best_it = None
    best_model = None
    best_loss = np.inf

    for epoch in range(epochs):
        pred = model(input_causes, input_targets)
        loss = sum([loss_fn(pred[i], Y[i]) for i in range(len(Y))])
        print("epoch {} loss {} :".format(epoch, loss / len(Y)))
        loss.backward()
        optimizer.step()
        model.zero_grad()

        mean_loss = loss / len(Y)

        if mean_loss < best_loss:
            best_loss = mean_loss
            best_it = epoch
            best_model = deepcopy(model)
        elif (epoch - best_it) == lookback:
            if verbose:
                print('Stopping early')
            break

    restore_parameters(model, best_model)

    return train_loss_list, model



def data_split(X, cause, target, timelag, device = device):
    input_cause = []
    input_target = []
    Y = []

    for i in range(len(X) - (timelag + 1)):
        input_cause.append(X[cause].values[i: i + timelag])
        input_target.append(X[target].values[i: i + timelag])
        Y.append([X[target][i + timelag + 1]])

    return torch.tensor(input_cause, device=device).float(), torch.tensor(input_target,device=device).float(), torch.tensor(Y, device=device).float()


In [77]:
import pandas as pd
df = pd.read_csv('C:/Users/chanyoung/Desktop/Neural-GC-master/lorenz_96_10_10_1000.csv')

In [78]:
X2d = df[['a','b']]
torch.manual_seed(1234)
input_cause, input_target, Y = data_split(X2d, 'a', 'b', 10)

In [81]:
model = RBFnet(input_cause.size()[0], 1, input_cause.size()[1])
print(model.networks[1].init_weight_cause)
print(model.networks[1].cause_std)
print(model.networks[1].target_std)
loss, best_model = train_rbf(model, input_cause, input_target, Y, 0.01, 120, device)
print(best_model.networks[1].init_weight_cause)
print(best_model.networks[1].cause_std)
print(model.networks[1].target_std)

Parameter containing:
tensor([0.3990, 0.5167, 0.0249, 0.9401, 0.9459, 0.7967, 0.4150, 0.8203, 0.2290,
        0.9096], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.7874], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.4503], device='cuda:0', requires_grad=True)
epoch 0 loss 27.376123428344727 :
epoch 1 loss 26.882343292236328 :
epoch 2 loss 26.395124435424805 :
epoch 3 loss 25.912689208984375 :
epoch 4 loss 25.43282699584961 :
epoch 5 loss 24.953168869018555 :
epoch 6 loss 24.471799850463867 :
epoch 7 loss 23.987049102783203 :
epoch 8 loss 23.497392654418945 :
epoch 9 loss 23.00132179260254 :
epoch 10 loss 22.49742889404297 :
epoch 11 loss 21.9847412109375 :
epoch 12 loss 21.46206283569336 :
epoch 13 loss 20.92828369140625 :
epoch 14 loss 20.38227653503418 :
epoch 15 loss 19.823266983032227 :
epoch 16 loss 19.250286102294922 :
epoch 17 loss 18.662784576416016 :
epoch 18 loss 18.060420989990234 :
epoch 19 loss 17.44284439086914 :
epoch 20 lo

In [92]:
input_cause.shape

torch.Size([989, 10])

In [84]:
h_0 = np.zeros((989,10))
from scipy.stats import f_oneway
list_ = []
for i in range(989):
    a = best_model.networks[i].init_weight_cause.detach().cpu().numpy()
    list_.append(a)
p_value = f_oneway(list_,h_0).pvalue


In [85]:
p_value

array([0.00000000e+000, 0.00000000e+000, 1.88810894e-060, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       3.56721508e-269, 0.00000000e+000])

In [89]:
def rbf_gradient(x, cluster, std):
    return (-2 * (x - cluster) / (std*std)) *(torch.exp(-(x - cluster) * (x - cluster) / 2*(std * std)))

def rbf(x, cluster, std):
    return torch.exp(-(x - cluster) * (x - cluster) / 2*(std * std))

In [90]:
for j in range(3):
    clt = best_model.networks[j].cause_clt
    std = best_model.networks[j].cause_std
    for i in range(len(input_cause[j]) - 2):
        rbf_grad = rbf_gradient(input_cause[j][i+1], clt, std)
        rbf_num_grad = (rbf(input_cause[j][i+2],clt,std) - rbf(input_cause[j][i], clt, std)) / (input_cause[j][i+2] - input_cause[j][i]) 
        print("rbf gradient :",rbf_grad)
        print("rbf_num_grad :", rbf_num_grad)
        print('----------------------------------------')
        
    print('next network')
    print()

rbf gradient : tensor([-3.0483], device='cuda:0', grad_fn=<MulBackward0>)
rbf_num_grad : tensor([-0.1652], device='cuda:0', grad_fn=<DivBackward0>)
----------------------------------------
rbf gradient : tensor([-0.0068], device='cuda:0', grad_fn=<MulBackward0>)
rbf_num_grad : tensor([-0.0524], device='cuda:0', grad_fn=<DivBackward0>)
----------------------------------------
rbf gradient : tensor([-1.0919e-06], device='cuda:0', grad_fn=<MulBackward0>)
rbf_num_grad : tensor([-0.0001], device='cuda:0', grad_fn=<DivBackward0>)
----------------------------------------
rbf gradient : tensor([-1.7484e-06], device='cuda:0', grad_fn=<MulBackward0>)
rbf_num_grad : tensor([-2.8808e-07], device='cuda:0', grad_fn=<DivBackward0>)
----------------------------------------
rbf gradient : tensor([-4.9685e-06], device='cuda:0', grad_fn=<MulBackward0>)
rbf_num_grad : tensor([-5.8298e-05], device='cuda:0', grad_fn=<DivBackward0>)
----------------------------------------
rbf gradient : tensor([-0.0035], de

In [93]:
loss_fn = nn.MSELoss(reduction='mean')

In [96]:
a = torch.zeros(2, 3)

In [97]:
b = torch.ones(2,3)

In [98]:
loss_fn(a,b)

tensor(1.)

In [99]:
input_cause

tensor([[ 0.0990,  2.3833,  6.1237,  ...,  0.7887, -3.0315, -3.0010],
        [ 2.3833,  6.1237,  8.7908,  ..., -3.0315, -3.0010, -1.9771],
        [ 6.1237,  8.7908,  8.6702,  ..., -3.0010, -1.9771, -1.6429],
        ...,
        [ 2.5767,  3.8119,  5.3918,  ...,  1.1669, -0.7102, -2.3594],
        [ 3.8119,  5.3918,  6.0065,  ..., -0.7102, -2.3594, -2.1374],
        [ 5.3918,  6.0065,  5.1861,  ..., -2.3594, -2.1374, -0.8038]],
       device='cuda:0')